In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

GENOMES = { "mm10" : "/users/kcochran/genomes/mm10_no_alt_analysis_set_ENCODE.fasta",
            "hg38" : "/users/kcochran/genomes/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta" }

ROOT = "/users/kcochran/projects/cs197_cross_species_domain_adaptation/"
DATA_DIR = ROOT + "data/"

SPECIES = ["mouse", "human"]

TFS = ["CTCF", "CEBPA", "HNF4A", "RXRA"]

TEST_DIR = "/users/tkanell/team_covariates/data/human/accessibility/peaks.bed"

In [12]:
import gzip
from collections import defaultdict
import random
import numpy as np
from pyfaidx import Fasta
from torch.utils.data import Dataset
#from torch.utils.data import Dataset
import pyBigWig


In [13]:
class Generator(Dataset):
    letter_dict = {
        'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1],
        'n':[0,0,0,0],'A':[1,0,0,0],'C':[0,1,0,0],'G':[0,0,1,0],
        'T':[0,0,0,1],'N':[0,0,0,0]}

    def __init__(self, species, tf, train_val_test,
                 seq_len = 2114, profile_len = 1000, return_labels = True):
        
        #assert train_val_test in ["train", "val", "test"]
        ## note: kelly will give you these files, but they are basically the same as normal peak files
        self.peakfile = TEST_DIR
        if train_val_test == "train":
            self.peakfile = PEAKS_DIR + species + "/" + tf + "/filtered_peaks_chr3toY.bed"
        elif train_val_test == "val":
            self.peakfile = PEAKS_DIR + species + "/" + tf + "/filtered_peaks_chr1.bed"
        # else:
        #     self.peakfile = PEAKS_DIR + species + "/" + tf + "/filtered_peaks_chr2.bed"
            
        #self.pos_bw = BIGWIGS_DIR + species + "/" + tf + "/final.pos.bigWig"
        #self.neg_bw = BIGWIGS_DIR + species + "/" + tf + "/final.neg.bigWig"
        self.prof_len = profile_len
        self.max_jitter = 0
        self.return_labels = return_labels
        
        #self.genome_file = GENOMES[species]
        self.seq_len = seq_len

        self.set_len()
        self.coords = self.get_coords()
        self.seqs_onehot = self.convert(self.coords)
        self.profiles, self.logcounts = self.get_profiles_and_logcounts(self.coords)


    def __len__(self):
        return self.len
    
    
    def set_len(self):
        with open(self.peakfile) as f:
            self.len = sum([1 for _ in f])


    def get_coords(self):
        with open(self.peakfile) as posf:
            coords_tmp = [line.split()[:3] for line in posf]  # expecting bed file format
        
        coords = []
        for coord in coords_tmp:
            chrom, start, end = coord[0], int(coord[1]), int(coord[2])
            window_start, window_end = expand_window(start, end,
                                                     self.seq_len + 2 * self.max_jitter)
            coords.append((coord[0], window_start, window_end))  # no strand consideration
        return coords
            

    def get_profiles_and_logcounts(self, coords):
        profiles = []
        logcounts = []

        with pyBigWig.open(self.pos_bw) as pos_bw_reader:
            with pyBigWig.open(self.neg_bw) as neg_bw_reader:
                for chrom, start, end in coords:
                    # need to trim the profile length to match model output size
                    # this is smaller than the input size bc of the receptive field
                    # and deconv layer kernel width
                    prof_start, prof_end = expand_window(start, end,
                                                 self.prof_len + 2 * self.max_jitter)
                    
                    pos_profile = np.array(pos_bw_reader.values(chrom, prof_start, prof_end))
                    pos_profile[np.isnan(pos_profile)] = 0
                    neg_profile = np.array(neg_bw_reader.values(chrom, prof_start, prof_end))
                    neg_profile[np.isnan(neg_profile)] = 0
                    profile = np.array([pos_profile, neg_profile])
                    
                    pos_logcount = np.log(np.sum(pos_profile) + 1)
                    neg_logcount = np.log(np.sum(neg_profile) + 1)
                    logcount = np.array([pos_logcount, neg_logcount])

                    profiles.append(profile)
                    logcounts.append(logcount)
                    
        profiles = np.array(profiles)
        logcounts = np.array(logcounts)
        return profiles, logcounts
                

    def convert(self, coords):
        seqs_onehot = []
        with Fasta(self.genome_file) as converter:
            for coord in coords:
                chrom, start, stop = coord
                assert chrom in converter
                seq = converter[chrom][start:stop].seq
                seq_onehot = np.array([self.letter_dict.get(x,[0,0,0,0]) for x in seq])
                seqs_onehot.append(seq_onehot)

        seqs_onehot = np.array(seqs_onehot)
        return seqs_onehot


    def __getitem__(self, batch_index):	
        # get coordinates
        onehot = self.seqs_onehot[batch_index]
        assert onehot.shape[0] > 0, onehot.shape

        onehot = torch.tensor(onehot, dtype=torch.float).permute(1, 0)
        
        if not self.return_labels:
            return onehot
        else:
            # get profiles and logcounts for the two strands
            profiles = self.profiles[batch_index]
            logcounts = self.logcounts[batch_index]

            profiles = torch.tensor(profiles, dtype=torch.float)
            logcounts = torch.tensor(logcounts, dtype=torch.float)
            return onehot, profiles, logcounts

In [14]:
def get_profiles_and_logcounts():
    profiles = []
    logcounts = []

    with pyBigWig.open(self.pos_bw) as pos_bw_reader:
        with pyBigWig.open(self.neg_bw) as neg_bw_reader:
            for chrom, start, end in coords:
                # need to trim the profile length to match model output size
                # this is smaller than the input size bc of the receptive field
                # and deconv layer kernel width
                prof_start, prof_end = expand_window(start, end,
                                             self.prof_len + 2 * self.max_jitter)

                pos_profile = np.array(pos_bw_reader.values(chrom, prof_start, prof_end))
                pos_profile[np.isnan(pos_profile)] = 0
                neg_profile = np.array(neg_bw_reader.values(chrom, prof_start, prof_end))
                neg_profile[np.isnan(neg_profile)] = 0
                profile = np.array([pos_profile, neg_profile])

                pos_logcount = np.log(np.sum(pos_profile) + 1)
                neg_logcount = np.log(np.sum(neg_profile) + 1)
                logcount = np.array([pos_logcount, neg_logcount])

                profiles.append(profile)
                logcounts.append(logcount)

    profiles = np.array(profiles)
    logcounts = np.array(logcounts)
    return profiles, logcounts

In [15]:
print("This working")
train_gen = Generator("human", "CTCF", "va")
print("second")
#source = Generator(train_species, tf, "val")

This working


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte